Load in required libraries, below.

In [43]:
# data 
import pandas as pd 
import numpy as np
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

%matplotlib inline

## SageMaker Resources

The below cell stores the SageMaker session and role (for creating estimators and models), and creates a default S3 bucket. After creating this bucket, locally stored data can be uploaded to S3.

In [37]:
# sagemaker
import boto3
import sagemaker
from sagemaker import get_execution_role

In [38]:
# SageMaker session and role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# default S3 bucket
bucket = sagemaker_session.default_bucket()

Here we retrieve the dataset of images and we upload it to S3


In [9]:
!wget -nc https://da-youtube-ml.s3.eu-central-1.amazonaws.com/wendy-cnn/frames/wendy_cnn_frames_data.zip
!unzip -qq -n wendy_cnn_frames_data.zip -d wendy_cnn_frames_data 
!rm wendy_cnn_frames_data.zip
prefix='cnn-wendy'

# upload to S3
input_data = sagemaker_session.upload_data(path='wendy_cnn_frames_data', bucket=bucket, key_prefix=prefix)
print(input_data)

--2020-10-18 21:09:16--  https://da-youtube-ml.s3.eu-central-1.amazonaws.com/wendy-cnn/frames/wendy_cnn_frames_data.zip
Resolving da-youtube-ml.s3.eu-central-1.amazonaws.com (da-youtube-ml.s3.eu-central-1.amazonaws.com)... 52.219.75.180
Connecting to da-youtube-ml.s3.eu-central-1.amazonaws.com (da-youtube-ml.s3.eu-central-1.amazonaws.com)|52.219.75.180|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3406848357 (3.2G) [application/zip]
Saving to: ‘wendy_cnn_frames_data.zip’

wendy_cnn_frames_da 100%[===================>]   3.17G  91.9MB/s    in 34s     

2020-10-18 21:09:50 (95.0 MB/s) - ‘wendy_cnn_frames_data.zip’ saved [3406848357/3406848357]

s3://sagemaker-eu-central-1-283211002347/cnn-wendy


After uploading images to S3, we can define and train the estimator


In [41]:
# import a PyTorch wrapper
from sagemaker.pytorch import PyTorch

# specify an output path
# prefix is specified above
output_path = 's3://{}/{}'.format(bucket, prefix)

# instantiate a pytorch estimator
estimator = PyTorch(entry_point='train.py',
                    source_dir='letsplay_classifier',
                    role=role,
                    framework_version='1.6',
                    train_instance_count=1,
                    train_instance_type='ml.p2.xlarge',
                    train_volume_size = 10,
                    output_path=output_path,
                    sagemaker_session=sagemaker_session,
                    hyperparameters={
                        'img-width': 128,
                        'img-height': 72,
                        'batch-size': 16,
                        'epochs': 10
                    })

## Train the Estimator

After instantiating the estimator, we train it with a call to `.fit()`. 

In [ ]:
%%time 
# train the estimator on S3 training data
estimator.fit({'train': input_data})

'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


2020-10-19 00:23:13 Starting - Starting the training job...
2020-10-19 00:23:14 Starting - Launching requested ML instances......
2020-10-19 00:24:17 Starting - Preparing the instances for training......
2020-10-19 00:25:29 Downloading - Downloading input data.................................
2020-10-19 00:31:05 Training - Downloading the training image..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-10-19 00:31:29,028 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2020-10-19 00:31:29,051 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.

2020-10-19 00:31:27 Training - Training image download completed. Training in progress.2020-10-19 00:31:35,273 sagemaker_pytorch_container.training INFO     Invoking user training script.
2020-10-19 00:31:35,778 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/opt/co

We set up a model that can predict the class of an image

### Deploy the trained model

We deploy our model to create a predictor. We'll use this to make predictions on our data and evaluate the model.

In [49]:
print(estimator.model_data)
# importing PyTorchModel
from sagemaker.pytorch import PyTorchModel

# Create a model from the trained estimator data
# And point to the prediction script
model = PyTorchModel(model_data=estimator.model_data,
                     role = role,
                     framework_version='1.6',
                     entry_point='predict.py',
                     source_dir='letsplay_classifier')

s3://sagemaker-eu-central-1-283211002347/cnn-wendy/pytorch-training-2020-10-19-00-23-12-908/output/model.tar.gz


In [51]:
%%time
# deploy and create a predictor
                
predictor = model.deploy(initial_instance_count=1, instance_type='ml.p2.xlarge')

'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.
Using already existing model: pytorch-inference-2020-10-19-04-13-18-987


-----------------!CPU times: user 38.2 s, sys: 5.96 s, total: 44.1 s
Wall time: 9min 17s


Now that the model is deployed, we check how it performs on our full dataset,
ensuring that the predictions make sense.


In [52]:
import os
import json
import torch
from PIL import Image

import torch.nn as nn

index = 0
loss_test = 0
acc_test = 0
count = 0
cur_data_dir = 'wendy_cnn_frames_data'
dirs = sorted(os.listdir(cur_data_dir))
avg_loss = 0
avg_acc = 0

# goes through all uncompressed images 
for dir in dirs:
    
    # as labels are sorted alphabetically, 
    labels = torch.empty(1, dtype=int)
    labels[0] = index
    
    curr_img_dir = os.path.join(cur_data_dir, dir)
    images = os.listdir(curr_img_dir)
    for image in images:
        curr_img = os.path.join(curr_img_dir, image)

        with open(curr_img, 'rb') as f:
            image_data = Image.open(f)
           
            prediction_from_ep = predictor.predict(image_data)  
            prediction = torch.FloatTensor(prediction_from_ep).unsqueeze(0)

            _, preds = torch.max(prediction.data, 1)
            loss = criterion(prediction, labels)

            loss_test += loss.data
            acc_test += torch.sum(preds == labels.data)
            count += 1
            avg_loss = torch.true_divide(loss_test, count)
            avg_acc = torch.true_divide(acc_test, count)
            if (count % 100 == 0):
                print("{} processed".format(count))
                print("Avg loss (test): {:.4f}".format(avg_loss))
                print("Avg acc (test): {:.4f}".format(avg_acc))
    index += 1
    
print("Avg loss (test): {:.4f}".format(avg_loss))
print("Avg acc (test): {:.4f}".format(avg_acc))


ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (0) from model with message "Your invocation timed out while waiting for a response from container model. Review the latency metrics for each container in Amazon CloudWatch, resolve the issue, and try again.". See https://eu-central-1.console.aws.amazon.com/cloudwatch/home?region=eu-central-1#logEventViewer:group=/aws/sagemaker/Endpoints/pytorch-inference-2020-10-19-04-13-18-987 in account 283211002347 for more information.

## Delete the Endpoint

Finally, I've add a convenience function to delete prediction endpoints after we're done with them. And if you're done evaluating the model, you should delete your model endpoint!

In [18]:
# Accepts a predictor endpoint as input
# And deletes the endpoint by name
def delete_endpoint(predictor):
        try:
            boto3.client('sagemaker').delete_endpoint(EndpointName=predictor.endpoint)
            print('Deleted {}'.format(predictor.endpoint))
        except:
            print('Already deleted: {}'.format(predictor.endpoint))

In [19]:
# delete the predictor endpoint 
delete_endpoint(predictor)

Deleted sagemaker-pytorch-2019-03-12-03-24-09-384
